In [ ]:
!pip install huggingface-hub==0.2.1 
!pip install ipykernel==6.6.0 
!pip install ipython==7.30.1
!pip install numpy==1.21.4 
!pip install numpy-base==1.21.2
!pip install pandas==1.3.5 
!pip install python==3.7.11 
!pip install pytorch==1.10.1
!pip install pytorch-lightning==1.5.6
!pip install pytorch-mutex==1.0 
!pip install tokenizers==0.10.3
!pip install torchaudio==0.10.1
!pip install torchmetrics==0.6.2
!pip install torchvision==0.11.2
!pip install tqdm==4.62.3
!pip install transformers==4.14.1
!pip install urllib3==1.26.7
!pip install vc==14.2
!pip install vs2015_runtime==14.27.29016

In [ ]:
import torch
from transformers import GPT2LMHeadModel

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
text = '한번 설정한 퇴직급여제도를 변경할 수 있나요? 있다면 그 절차는 무엇인가요?'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

In [ ]:
def chatbot(text):
  input_ids = tokenizer.encode(text)
  gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
  generated = tokenizer.decode(gen_ids[0,:].tolist())
  return   print('사용자: {}'.format(text)),print('\n'), print('답변: {}'.format(generated).strip(("None, None, None")))

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
data = pd.read_csv("naver_news_all2.csv")
data.head()

In [ ]:
data2 = pd.read_csv("news_double.csv")
data2.head()

In [ ]:
chatbot_data2 = data2[['article', 'sub']]

In [ ]:
chatbot_data = data[['article', 'subject']]

In [ ]:
chatbot_data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = chatbot_data
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = chatbot_data2
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, )

In [ ]:
model.to(device)
model.train()

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [ ]:
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

# 뉴스 크롤링 학습 않음 

In [ ]:
text = "한번 설정한 퇴직급여제도를 변경할 수 있나요? 있다면 그 절차는 무엇인가요?"
chatbot(text)

# naver_news_all2 학습

In [ ]:
text = "한번 설정한 퇴직급여제도를 변경할 수 있나요? 있다면 그 절차는 무엇인가요?"
chatbot(text)

# naver_double 학습

In [ ]:
text = "한번 설정한 퇴직급여제도를 변경할 수 있나요? 있다면 그 절차는 무엇인가요?"
chatbot(text)